In [37]:
from langchain.memory import ConversationKGMemory
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(temperature=0.1)

memory = ConversationKGMemory(
    llm=llm,
    return_messages=True,
)

def add_message(input,output):
    memory.save_context({"input":input},{"output":output})

add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")

In [38]:
memory.load_memory_variables({"input":"who is Nicolas"})

{'history': [SystemMessage(content='On Nicolas: Nicolas lives in South Korea.', additional_kwargs={}, response_metadata={})]}

In [39]:
add_message("니콜라스는 김치를 좋아환다.", "Wow that is so cool!!!")

In [40]:
memory.load_memory_variables({"input":"니콜라스가 좋아하는 음식은?"})

{'history': [SystemMessage(content='On Nicolas: Nicolas lives in South Korea. Nicolas likes kimchi.', additional_kwargs={}, response_metadata={})]}